### 크롤링으로 전자장비와기기 업종 리스트 가져오기

In [88]:
#com_list = []
import requests
from bs4 import BeautifulSoup
import pandas as pd

#com_list가 전자장비와기기 업종 회사이름
tickers = {}
url = "https://finance.naver.com/sise/sise_group_detail.nhn?type=upjong&no=197"
rq = requests.get(url)
html = rq.text

soup = BeautifulSoup(html,"html.parser")
rows = soup.select('td > div')

for r in rows[1:]:
    tickers[r.select('a')[0]['href'][20:]] = r.text
a = [k for k in tickers.keys() if tickers[k][-2]=='우' or tickers[k][-3]=='우']
# 우량주 종목 제외    
for i in range(len(a)):
    del tickers[a[i]]

code = list(tickers.keys())
com = list(tickers.values())
annual_date = ['2015/12', '2016/12', '2017/12', '2018/12', '2019/12', '2020/12','2021/12', '2022/12']

In [89]:
tickers

{'066900': '디에이피 *',
 '085670': '뉴프렉스 *',
 '007660': '이수페타시스 ',
 '125210': '아모그린텍 *',
 '004270': '남성 ',
 '099520': 'ITX엠투엠 *',
 '068940': '아이씨케이 *',
 '082210': '옵트론텍 *',
 '051370': '인터플렉스 *',
 '101390': '아이엠 *',
 '007810': '코리아써키트 ',
 '072990': '에이치시티 *',
 '143160': '아이디스 *',
 '090460': '비에이치 *',
 '046310': '백금T&A; *',
 '187220': '디티앤씨 *',
 '008060': '대덕 ',
 '052020': '에스티큐브 *',
 '140860': '파크시스템스 *',
 '131400': '액트 *',
 '033240': '자화전자 ',
 '008110': '대동전자 ',
 '038460': '바이오스마트 *',
 '011230': '삼화전자 ',
 '262260': '에이프로 *',
 '353200': '대덕전자 ',
 '078600': '대주전자재료 *',
 '222800': '심텍 *',
 '250060': '모비스 *',
 '236200': '슈프리마 *',
 '036710': '심텍홀딩스 *',
 '053450': '세코닉스 *',
 '017900': '광전자 ',
 '272290': '이녹스첨단소재 *',
 '093520': '매커스 *',
 '057540': '옴니시스템 *',
 '083640': '인콘 *',
 '011070': 'LG이노텍 ',
 '092300': '현우산업 *',
 '069510': '에스텍 *',
 '033200': '모아텍 *',
 '004770': '써니전자 ',
 '017250': '인터엠 *',
 '009150': '삼성전기 ',
 '336370': '두산솔루스 ',
 '096040': '이트론 *',
 '091340': 'S&K;폴리텍 *',
 '052330': '코텍 

### 네이버 재무제표 크롤링

In [90]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
import pandas as pd
import time
import urllib.request
from selenium.webdriver import Chrome
import json
import re
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import datetime as dt

In [91]:
browser  = Chrome()
browser.maximize_window()

def ttt(xx):
    name = xx
    base_url = 'https://finance.naver.com/item/coinfo.nhn?code='+ name + '&target=finsum_more'
    return base_url

In [92]:
def stock_crawler(code):
    #code = 종목번호
    name = code
    base_url = 'https://finance.naver.com/item/coinfo.nhn?code='+ name + '&target=finsum_more'
    
    browser.get(base_url)
    #frmae구조 안에 필요한 데이터가 있기 때문에 해당 데이터를 수집하기 위해서는 frame구조에 들어가야한다.
    browser.switch_to_frame(browser.find_element_by_id('coinfo_cp'))
    
    #재무제표 "연간" 클릭하기
    browser.find_elements_by_xpath('//*[@class="schtab"][1]/tbody/tr/td[3]')[0].click()

    html0 = browser.page_source
    html1 = BeautifulSoup(html0,'html.parser')
    
    #기업명 뽑기
    title0 = html1.find('head').find('title').text
    #print(title0.split('-')[-1])
    
    html22 = html1.find('table',{'class':'gHead01 all-width','summary':'주요재무정보를 제공합니다.'})
    
    #date scrapy
    thead0 = html22.find('thead')
    tr0 = thead0.find_all('tr')[1]
    th0 = tr0.find_all('th')
    
    date = []
    for i in range(len(th0)):
        date.append(''.join(re.findall('[0-9/]',th0[i].text)))
    
    #columns scrapy
    tbody0 = html22.find('tbody')
    tr0 = tbody0.find_all('tr')
    
    col = []
    for i in range(len(tr0)):

        if '\xa0' in tr0[i].find('th').text:
            tx = re.sub('\xa0','',tr0[i].find('th').text)
        else:
            tx = tr0[i].find('th').text

        col.append(tx)
    
    #main text scrapy
    td = []
    for i in range(len(tr0)):
        td0 = tr0[i].find_all('td')
        td1 = []
        for j in range(len(td0)):
            if td0[j].text == '':
                td1.append('0')
            else:
                td1.append(td0[j].text)

        td.append(td1)
    
    td2 = list(map(list,zip(*td)))
    
    return pd.DataFrame(td2,columns = col,index = date).T

In [13]:
samsung.columns

Index(['매출액', '영업이익', '영업이익(발표기준)', '세전계속사업이익', '당기순이익', '당기순이익(지배)',
       '당기순이익(비지배)', '자산총계', '부채총계', '자본총계', '자본총계(지배)', '자본총계(비지배)', '자본금',
       '영업활동현금흐름', '투자활동현금흐름', '재무활동현금흐름', 'CAPEX', 'FCF', '이자발생부채', '영업이익률',
       '순이익률', 'ROE(%)', 'ROA(%)', '부채비율', '자본유보율', 'EPS(원)', 'PER(배)',
       'BPS(원)', 'PBR(배)', '현금DPS(원)', '현금배당수익률', '현금배당성향(%)', '발행주식수(보통주)'],
      dtype='object')

In [93]:
# array로 정의
#df_NI(Net Income) : 당기순이익 데이터프레임
df_PER,df_PBR,df_ROE,df_ROA,df_EPS,df_NI = np.array([]),np.array([]),np.array([]),np.array([]),np.array([]),np.array([])

for c in code:
    df = stock_crawler(c)
    df_PER = np.hstack([df_PER,df.loc['PER(배)'].values])
    df_PBR = np.hstack([df_PBR,df.loc['PBR(배)'].values])
    df_EPS = np.hstack([df_EPS,df.loc['EPS(원)'].values])
    df_ROE = np.hstack([df_ROE,df.loc['ROE(%)'].values])
    df_ROA = np.hstack([df_ROA,df.loc['ROA(%)'].values])
    df_NI = np.hstack([df_NI,df.loc['당기순이익'].values])    

C:\Users\skxod\.conda\envs\keras\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: use driver.switch_to.frame instead
  


In [50]:
# numpy -> pandas 데이터프레임으로 변환하는 사용자 정의 함수
def trans_df(ar):
    ar.resize((int(len(ar)/8),8))
    df = pd.DataFrame(data = ar,index=com,columns = annual_date)
    return df 

In [61]:
# 데이터프레임으로 변환
df_PER = trans_df(df_PER)
df_PBR = trans_df(df_PBR)
df_EPS = trans_df(df_EPS)
df_ROE = trans_df(df_ROE)
df_ROA = trans_df(df_ROA)
df_NI = trans_df(df_NI)

In [74]:
# 지표별로 랭크 구하여 합산하기
df_rank = df_PER.rank(method='min')+df_PBR.rank(method='min')+df_EPS.rank(method='max')+df_ROE.rank(method='max')+df_ROA.rank(method='max')
+df_NI.rank(method='max')

연도 입력 2019


In [78]:
# 선택년도에 따라 순위 정렬
year = input('연도 입력 ')
sc = year+'/12'
df_rank.sort_values(by=sc,inplace=True)

연도 입력 2019


In [86]:
top10 = list(df_rank.index)[:10]

In [87]:
# matplotlib rebuild
import matplotlib as mpl
#mpl.font_manager._rebuild()
# matplotlib 환경 설정
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import FinanceDataReader as fdr

plt.rcParams["font.family"] = 'nanummyeongjo'
plt.rcParams["axes.grid"] = True
plt.rcParams["figure.figsize"] = (12,6)
plt.rcParams["axes.formatter.useoffset"] = False
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams["axes.formatter.limits"] = -10000, 10000

['자화전자 ',
 '대덕전자 ',
 '세코닉스 *',
 '모아텍 *',
 '심텍홀딩스 *',
 '이그잭스 *',
 '비덴트 *',
 '인터엠 *',
 '아이디스홀딩스 *',
 '남성 ']